In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
from sympy import *
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline
init_printing()

In [2]:
Variables=symbols("x1 y1 x2 y2")
x1,y1,x2,y2 = Variables
Time=symbols("t")
t=Time
Parameters = symbols("alpha beta gamma delta w a0 a1 omega epsilon")
alpha,beta,gamma,delta,w,a0,a1,omega,epsilon = Parameters

### Duffing system

In [3]:
F=Matrix([y1, gamma*cos(omega*t) - delta*y1 - beta*x1 - alpha*x1**3,
          y2, gamma*cos(omega*t) - delta*y2 - beta*x2 - alpha*x2**3])
F

⎡                y₁                ⎤
⎢                                  ⎥
⎢      3                           ⎥
⎢- α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(ω⋅t)⎥
⎢                                  ⎥
⎢                y₂                ⎥
⎢                                  ⎥
⎢      3                           ⎥
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(ω⋅t)⎦

In [4]:
zeta=Matrix([0,0,
             0,0])
#zeta=eps*zeta
zeta

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

### Synchrony

In [5]:
phi=Matrix([(x2-x1*(a0 + a1*sin(w*t)))])
phi

[-x₁⋅(a₀ + a₁⋅sin(t⋅w)) + x₂]

In [6]:
N=simulation.VectorDivergence(phi,Variables)
N

[-a₀ - a₁⋅sin(t⋅w)  0  1  0]

In [7]:
Ndot = diff(N,Time)
Ndot

[-a₁⋅w⋅cos(t⋅w)  0  0  0]

In [8]:
on_manifold = [(x2,x1*(a0+a1*sin(w*t)))]

## Invariance

In [9]:
# NF=(N*(F+zeta)).subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
# simplify(NF)
NF=N*(F+zeta)
NF
#simplify(NF)

[y₁⋅(-a₀ - a₁⋅sin(t⋅w)) + y₂]

In [10]:
dPhi=diff(phi,Time)
dPhi

[-a₁⋅w⋅x₁⋅cos(t⋅w)]

In [11]:
eqn=(NF+dPhi)
simplify(eqn)

[-a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(a₀ + a₁⋅sin(t⋅w)) + y₂]

In [12]:
# This is working only because of the nature of the N matrix - be careful
zeta[2]=-(eqn)[0]
#zeta[3]=-(eqn)[1]

simplify(zeta)

⎡                      0                      ⎤
⎢                                             ⎥
⎢                      0                      ⎥
⎢                                             ⎥
⎢a₁⋅w⋅x₁⋅cos(t⋅w) + y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂⎥
⎢                                             ⎥
⎣                      0                      ⎦

In [13]:
zeta=zeta.subs(on_manifold)
zeta

⎡                      0                       ⎤
⎢                                              ⎥
⎢                      0                       ⎥
⎢                                              ⎥
⎢a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(-a₀ - a₁⋅sin(t⋅w)) - y₂⎥
⎢                                              ⎥
⎣                      0                       ⎦

In [14]:
NF=N*(F+zeta)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(NF)
simplify(NF+dPhi).subs(on_manifold)

[0]

In [15]:
NZ=N*zeta
NZ

[a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(-a₀ - a₁⋅sin(t⋅w)) - y₂]

## Stability

In [23]:
chi=Matrix([0,0,epsilon+sin(w*t),0])*phi

#chi=(epsilon1*sin(b*t)+epsilon2)*chi 
#chi=epsilon*chi
#epsilon=1
chi

⎡                     0                      ⎤
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢(ε + sin(t⋅w))⋅(-x₁⋅(a₀ + a₁⋅sin(t⋅w)) + x₂)⎥
⎢                                            ⎥
⎣                     0                      ⎦

### stability criterion

### directional

In [24]:
simplify((Ndot+simulation.VectorDivergence(N*(F+zeta+chi),Variables))*N.transpose())

⎡                      2                          ⎤
⎣ε + (a₀ + a₁⋅sin(t⋅w)) ⋅(ε + sin(t⋅w)) + sin(t⋅w)⎦

### separational

In [26]:
simplify(N*simulation.VectorDivergence(F+chi+zeta,Variables)*N.transpose())

[ε - (a₀ + a₁⋅sin(t⋅w))⋅(a₁⋅w⋅cos(t⋅w) - (a₀ + a₁⋅sin(t⋅w))⋅(ε + sin(t⋅w))) + 
sin(t⋅w)]

-------------------------